In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
    model="gpt-3.5-turbo",
)

examples = [
    {
        "input": "Inception",
        "output": "영화 제목: Inception\n감독: Christopher Nolan\n주요 출연진: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page\n예산: $160 million\n흥행 수익: $836.8 million\n장르: Science Fiction, Thriller\n간략한 줄거리: 꿈속에서 다른 사람의 무의식을 해킹하는 전문가가 마지막 미션으로 한 기업가의 마음속에 아이디어를 심는 임무를 맡는다."
    },
    {
        "input": "Titanic",
        "output": "영화 제목: Titanic\n감독: James Cameron\n주요 출연진: Leonardo DiCaprio, Kate Winslet, Billy Zane\n예산: $200 million\n흥행 수익: $2.2 billion\n장르: Romance, Drama\n간략한 줄거리: 1912년, RMS 타이타닉호에서 만난 두 젊은 연인이 계급 차이를 극복하고 사랑을 키워 나가지만 비극적인 결말을 맞이한다."
    }
]

example_template = PromptTemplate(
    input_variables=["input", "output"],
    template="질문: {input}\n답변: {output}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="다음은 영화 제목을 입력하면 해당 영화의 정보를 제공하는 예시입니다:",
    suffix="질문: {movie_name}\n답변:",
    input_variables=["movie_name"]
)

chain = few_shot_prompt | llm
chain.invoke({"movie_name": "Interstellar"})


영화 제목: Interstellar
감독: Christopher Nolan
주요 출연진: Matthew McConaughey, Anne Hathaway, Jessica Chastain
예산: $165 million
흥행 수익: $677.5 million
장르: Science Fiction, Drama
간략한 줄거리: 미래 지구에서 식량 부족으로 인류가 멸종 위기에 처한 가운데, 한 우주 탐사대가 새로운 행성을 찾기 위해 우주로 떠난다.

AIMessage(content='영화 제목: Interstellar\n감독: Christopher Nolan\n주요 출연진: Matthew McConaughey, Anne Hathaway, Jessica Chastain\n예산: $165 million\n흥행 수익: $677.5 million\n장르: Science Fiction, Drama\n간략한 줄거리: 미래 지구에서 식량 부족으로 인류가 멸종 위기에 처한 가운데, 한 우주 탐사대가 새로운 행성을 찾기 위해 우주로 떠난다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-329245c5-49e3-478c-87e1-876d715e86a2-0')